In [1]:
%matplotlib inline

In [3]:
import pandas
import requests
import StringIO

ImportError: No module named 'stringio'

In [ ]:
cols = ['WTEQ', 'SNWD', 'PREC', 'TOBS',] # 'TMAX', 'TMIN', 'TAVG']
# water equivalent, snow depth, precipitation, observed temperature,

In [ ]:
colstring = ','.join('{}::value'.format(name) for name in cols)
colstring

In [ ]:
fmt = ('http://wcc.sc.egov.usda.gov/reportGenerator/view_csv/'
       'customSingleStationReport/hourly/651:OR:SNTL/{start},{end}/{columns}')

In [ ]:
url = fmt.format(start='2014-12-31', end='2015-01-31', columns=colstring)
url

In [ ]:
r = requests.get(url)
print(r.text)

In [ ]:
df = pandas.read_csv(StringIO.StringIO(r.text), comment='#')
df = df.set_index('Date')
df

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
axes = df.plot(subplots=True, figsize=(6,12), legend=True)
plt.subplots_adjust(hspace=0)
axes[-1].axhline(32, c='k', ls='--')

In [ ]:
import matplotlib
matplotlib.__version__

# compare against NWAC ratings for mt Hood

In [ ]:
import pickle

In [ ]:
with open('nwac_results_mthood.pickle') as f:
    results = pickle.load(f)

In [ ]:
df2 = pandas.DataFrame(results)
df2['utc'] = pandas.to_datetime(df2['date'])
df2 = df2.sort_values(by='utc').set_index('utc')
df2

In [ ]:
ratings = ['No Rating', 'Low', 'Moderate', 'Considerable', 'High', 'Extreme']

In [ ]:
ratdict = {rat:i for i,rat in enumerate(ratings)}

df3 = df2.reset_index().replace({'treeline-above':ratdict,
            'treeline-below':ratdict,
            'treeline-near':ratdict,}
                 ).set_index('utc')

In [ ]:
series = df3['treeline-above'].resample('1H', fill_method='pad')
series.name = 'NWAC Warning level'
dfx = df.join(series, how='inner')

In [ ]:
axes = dfx.plot(subplots=True, figsize=(6,12), legend=True)
plt.subplots_adjust(hspace=0)

axes[-2].axhline(32, c='k', ls='--')
axes[-1].set_ylim(-0,5);
axes[-1].set_yticklabels(ratings)
axes[-1].lines[0].set_ms('o')

plt.savefig('jan2015.pdf')